In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)
# retriever은 vectorstoe에서 가져와도 되지만 invoke등을 통해 가져올 수 도 있음
chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")

--- Logging error ---
Traceback (most recent call last):
  File "/Users/jeongdaegyu/Documents/gpt_pratice_dk/env/lib/python3.11/site-packages/langchain/callbacks/manager.py", line 2072, in _configure
    handler = LangChainTracer(project_name=tracer_project)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jeongdaegyu/Documents/gpt_pratice_dk/env/lib/python3.11/site-packages/langchain/callbacks/tracers/langchain.py", line 97, in __init__
    self.client = client or get_client()
                            ^^^^^^^^^^^^
  File "/Users/jeongdaegyu/Documents/gpt_pratice_dk/env/lib/python3.11/site-packages/langchain/callbacks/tracers/langchain.py", line 55, in get_client
    _CLIENT = Client()
              ^^^^^^^^
  File "/Users/jeongdaegyu/Documents/gpt_pratice_dk/env/lib/python3.11/site-packages/langsmith/client.py", line 272, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/Users/jeongdaegyu/Documents/gpt_pratice_dk/env/lib/pytho

'Victory Mansions is the building where Winston Smith lives. It is a run-down, dilapidated apartment complex that is part of the city of Airstrip One in Oceania. The building is described as having a hallway that smells of boiled cabbage and old rag mats. The elevator is often not working due to power outages, and there is a large poster of the face of Big Brother watching over the residents. The building is in disrepair, with rotting nineteenth-century houses, patched windows, and sagging garden walls surrounding it. Overall, Victory Mansions is portrayed as a grim and depressing place to live.'